In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb
import os
os.environ["WANDB_API_KEY"] ="61e79ad90458f5faea9605e7805f1f26268f0483"
os.environ["WANDB_SILENT"] = "true"
# Initialize a new wandb run
wandb.init(project="simple_mnist_example")

# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Training loop
for epoch in range(5):  # Run for 5 epochs
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print statistics and log them to wandb
        running_loss += loss.item()
        if i % 100 == 99:  # Every 100 mini-batches
            print(f"[Epoch {epoch+1}, Batch {i+1}] Loss: {running_loss / 100}")
            
            # Log the loss to wandb
            wandb.log({"Loss": running_loss / 100})
            
            running_loss = 0.0

# Finish the wandb run
wandb.finish()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.15MB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 151kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.26MB/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.20MB/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

[Epoch 1, Batch 100] Loss: 1.0356971633434295
[Epoch 1, Batch 200] Loss: 0.42296799302101135
[Epoch 1, Batch 300] Loss: 0.3749110445380211
[Epoch 1, Batch 400] Loss: 0.3723298877477646
[Epoch 1, Batch 500] Loss: 0.3384709666669369
[Epoch 1, Batch 600] Loss: 0.30464345283806327
[Epoch 1, Batch 700] Loss: 0.30988201215863226
[Epoch 1, Batch 800] Loss: 0.26651987075805667
[Epoch 1, Batch 900] Loss: 0.26269188314676284
[Epoch 2, Batch 100] Loss: 0.25308925986289976
[Epoch 2, Batch 200] Loss: 0.2130228051543236
[Epoch 2, Batch 300] Loss: 0.21778864972293377
[Epoch 2, Batch 400] Loss: 0.19754103414714336
[Epoch 2, Batch 500] Loss: 0.186408255584538
[Epoch 2, Batch 600] Loss: 0.16243046775460243
[Epoch 2, Batch 700] Loss: 0.1800655871257186
[Epoch 2, Batch 800] Loss: 0.1673584782332182
[Epoch 2, Batch 900] Loss: 0.17949155814945697
[Epoch 3, Batch 100] Loss: 0.13708874419331551
[Epoch 3, Batch 200] Loss: 0.138647576868

In [7]:
# Required imports
import torch
from matplotlib import pyplot as plt
import numpy as np
from collections import Counter
import wandb
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def load_model(model_name='gpt2', device='cuda'):
    """
    Load the model, tokenizer and word embeddings with proper configuration
    """
    model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    
    # Set proper padding configuration
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id
    
    word_embeddings = model.transformer.wte.weight
    return model, word_embeddings, tokenizer

def closest_tokens(embedding, word_embeddings, tokenizer, n=1):
    """
    Find the closest tokens to a given embedding
    """
    with torch.no_grad():  # No need to track gradients here
        distances = torch.sqrt(((word_embeddings - embedding.unsqueeze(0))**2).sum(-1))
        closest_indices = distances.argsort()[:n]
        closest_words = [tokenizer.decode([idx.item()]) for idx in closest_indices]
        return closest_words, closest_indices, distances[closest_indices], embedding

def model_emb(model, input_embeddings, word_embeddings, output_len):
    """
    Get model outputs for given input embeddings
    """
    # Create attention mask (all 1s since we're not padding)
    attention_mask = torch.ones(
        (input_embeddings.shape[0], input_embeddings.shape[1]), 
        device=input_embeddings.device
    )
    
    outputs = model(
        inputs_embeds=input_embeddings,
        attention_mask=attention_mask
    )
    
    logits = outputs.logits[:, :output_len, :]
    
    # Calculate perplexity
    with torch.no_grad():
        log_probs = torch.log_softmax(logits, dim=-1)
        perplexity = -log_probs.mean(dim=-1)
    
    return logits, input_embeddings, perplexity

def optimize_input(
    model,
    word_embeddings,
    tokenizer,
    device,
    input_len=10,
    target_output=' world',
    batch_size=20,
    epochs=100,
    lr=0.1,
    dist_reg=0.1,
    verbose=True
):
    """
    Optimize input embeddings to generate desired target output
    """
    print(f'Optimizing input of length {input_len} to maximize output logits for "{target_output}"')
    
    # Get target output indices
    output_ix = tokenizer.encode(target_output, return_tensors='pt')[0].to(device)
    output_len = output_ix.shape[0]
    
    # Normalize word embeddings
    word_embeddings = word_embeddings / torch.sqrt(torch.sum(word_embeddings**2, dim=-1, keepdim=True))
    
    # Initialize input embeddings
    start_input = word_embeddings[torch.randperm(word_embeddings.shape[0])[:input_len * batch_size]].reshape(
        batch_size, input_len, -1)
    input_embeds = torch.nn.Parameter(start_input.to(device), requires_grad=True)
    
    # Setup optimizer
    optimizer = torch.optim.Adam([input_embeds], lr=lr, eps=0.0001)
    
    optimized_inputs = set()
    
    try:
        for epoch in range(epochs):
            # Normalize input embeddings
            norm_input = input_embeds / torch.sqrt(torch.sum(input_embeds**2, dim=-1, keepdim=True))
            
            # Get model outputs
            logits, emb, perp = model_emb(model, norm_input, word_embeddings, output_len)
            probs = torch.softmax(logits, dim=-1)
            
            # Calculate target probabilities
            target_probs = probs[:, torch.arange(output_len), output_ix]
            
            # Calculate token distances
            token_distances = []
            closest_indices = []
            
            for batch in norm_input:
                batch_distances = []
                batch_indices = []
                for embedding in batch:
                    _, idx, dist, _ = closest_tokens(embedding, word_embeddings, tokenizer)
                    batch_distances.append(dist)
                    batch_indices.append(idx)
                token_distances.append(torch.stack(batch_distances))
                closest_indices.append(torch.stack(batch_indices))
            
            token_distances = torch.stack(token_distances).squeeze(-1)
            closest_indices = torch.stack(closest_indices).squeeze(-1)
            
            # Calculate losses
            prob_loss = -torch.log(target_probs).mean()
            dist_loss = token_distances.mean() * dist_reg
            total_loss = prob_loss + dist_loss
            
            # Generate outputs using closest tokens
            with torch.no_grad():
                attention_mask = torch.ones((batch_size, input_len), device=device)
                model_outputs = model.generate(
                    closest_indices,
                    max_length=output_len + input_len,
                    pad_token_id=model.config.pad_token_id,
                    attention_mask=attention_mask
                )
            
            # Check for successful generations
            for batch_idx in range(batch_size):
                output_text = tokenizer.decode(model_outputs[batch_idx][input_len:])
                if target_output in output_text:
                    full_text = tokenizer.decode(model_outputs[batch_idx])
                    if full_text not in optimized_inputs:
                        optimized_inputs.add(full_text)
                        if verbose:
                            print(f"\nFound new solution (Epoch {epoch}):")
                            print(f"Input: {tokenizer.decode(model_outputs[batch_idx][:input_len])}")
                            print(f"Output: {output_text}")
            
            # Optimization step
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            
            # Log progress
            if verbose and (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss.item():.4f}')
                
            # Optional: Log to wandb
            wandb.log({
                'total_loss': total_loss.item(),
                'prob_loss': prob_loss.item(),
                'dist_loss': dist_loss.item(),
                'num_solutions': len(optimized_inputs)
            })
            
    except KeyboardInterrupt:
        print("\nOptimization interrupted by user")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise
    
    return optimized_inputs

# Example usage in notebook cells:

# Cell 1: Initialize wandb (optional)
def init_wandb(run_name="test-run"):
    try:
        wandb.init(project='model-optimization', name=run_name)
    except Exception as e:
        print(f"Warning: Could not initialize wandb: {str(e)}")
        print("Continuing without wandb logging...")

# Cell 2: Load model and dependencies
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, word_embeddings, tokenizer = load_model(device=device)

# Cell 3: Run optimization
init_wandb()  # Comment this out if you don't want to use wandb
results = optimize_input(
    model=model,
    word_embeddings=word_embeddings,
    tokenizer=tokenizer,
    device=device,
    input_len=10,
    target_output=' world',
    batch_size=20,
    epochs=100,
    lr=0.01,
    dist_reg=0.1,
    verbose=True
)

Optimizing input of length 10 to maximize output logits for " world"
Epoch 10/100, Loss: 5.7270
Epoch 20/100, Loss: 4.9347
Epoch 30/100, Loss: 4.6405
Epoch 40/100, Loss: 4.5578
Epoch 50/100, Loss: 4.5343
Epoch 60/100, Loss: 4.5267

Optimization interrupted by user


In [14]:
import torch.optim.lr_scheduler as lr_scheduler
def optimize_input(
    model,
    word_embeddings,
    tokenizer,
    device,
    input_len=10,
    target_output=' world',
    batch_size=20,
    epochs=100,
    lr=0.01,
    dist_reg=0.05,
    verbose=True
):
    print(f'Optimizing input of length {input_len} to maximize output logits for "{target_output}"')
    
    output_ix = tokenizer.encode(target_output, return_tensors='pt')[0].to(device)
    output_len = output_ix.shape[0]
    
    word_embeddings = word_embeddings / torch.sqrt(torch.sum(word_embeddings**2, dim=-1, keepdim=True))
    
    vocab_size = word_embeddings.shape[0]
    indices = torch.randperm(vocab_size)[:input_len * batch_size]
    start_input = word_embeddings[indices].reshape(batch_size, input_len, -1)
    input_embeds = torch.nn.Parameter(start_input.to(device), requires_grad=True)
    
    optimizer = torch.optim.AdamW([input_embeds], lr=lr, weight_decay=0.01)
    scheduler = torch.optim.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
    
    optimized_inputs = set()
    best_loss = float('inf')
    patience = 0
    max_patience = 20
    
    try:
        for epoch in range(epochs):
            norm_input = input_embeds / (torch.norm(input_embeds, dim=-1, keepdim=True) + 1e-6)
            
            logits, emb, perp = model_emb(model, norm_input, word_embeddings, output_len)
            
            log_probs = torch.log_softmax(logits, dim=-1)
            target_log_probs = log_probs[:, torch.arange(output_len), output_ix]
            
            position_weights = torch.linspace(1.0, 0.5, output_len).to(device)
            prob_loss = -(target_log_probs * position_weights).mean()
            
            token_distances = []
            closest_indices = []
            
            for batch in norm_input:
                batch_distances = []
                batch_indices = []
                for embedding in batch:
                    _, idx, dist, _ = closest_tokens(embedding, word_embeddings, tokenizer)
                    batch_distances.append(dist)
                    batch_indices.append(idx)
                token_distances.append(torch.stack(batch_distances))
                closest_indices.append(torch.stack(batch_indices))
            
            token_distances = torch.stack(token_distances).squeeze(-1)
            closest_indices = torch.stack(closest_indices).squeeze(-1)
            
            # Convert unique penalty to tensor
            unique_tokens = torch.tensor([len(set(closest_indices[b].tolist())) for b in range(batch_size)], 
                                      device=device, dtype=torch.float)
            unique_penalty = (1.0 - unique_tokens / input_len).mean()
            
            dist_loss = token_distances.mean() * dist_reg
            diversity_loss = unique_penalty * 0.1
            total_loss = prob_loss + dist_loss + diversity_loss
            
            if total_loss.item() < best_loss:
                best_loss = total_loss.item()
                patience = 0
            else:
                patience += 1
            
            if patience >= max_patience:
                print(f"\nEarly stopping at epoch {epoch} due to no improvement")
                break
            
            with torch.no_grad():
                attention_mask = torch.ones((batch_size, input_len), device=device)
                model_outputs = model.generate(
                    closest_indices,
                    max_length=output_len + input_len,
                    pad_token_id=model.config.pad_token_id,
                    attention_mask=attention_mask,
                    do_sample=True,
                    top_p=0.9,
                    temperature=0.7
                )
            
            for batch_idx in range(batch_size):
                output_text = tokenizer.decode(model_outputs[batch_idx][input_len:])
                if target_output in output_text:
                    full_text = tokenizer.decode(model_outputs[batch_idx])
                    if full_text not in optimized_inputs:
                        optimized_inputs.add(full_text)
                        if verbose:
                            print(f"\nFound new solution (Epoch {epoch}):")
                            print(f"Input: {tokenizer.decode(model_outputs[batch_idx][:input_len])}")
                            print(f"Output: {output_text}")
            
            optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_([input_embeds], max_norm=1.0)
            optimizer.step()
            scheduler.step(total_loss)
            
            if verbose and (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss.item():.4f} '
                      f'(Prob: {prob_loss.item():.4f}, Dist: {dist_loss.item():.4f}, '
                      f'Div: {diversity_loss.item():.4f})')
                
            wandb.log({
                'total_loss': total_loss.item(),
                'prob_loss': prob_loss.item(),
                'dist_loss': dist_loss.item(),
                'diversity_loss': diversity_loss.item(),
                'num_solutions': len(optimized_inputs),
                'learning_rate': optimizer.param_groups[0]['lr']
            })
            
    except KeyboardInterrupt:
        print("\nOptimization interrupted by user")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise
    
    return optimized_inputs

In [15]:
results = optimize_input(
    model=model,
    word_embeddings=word_embeddings,
    tokenizer=tokenizer,
    device=device,
    input_len=10,
    target_output=' world',
    batch_size=20,
    epochs=100,
    lr=0.01,  # Using lower learning rate
    dist_reg=0.05,  # Lower distance regularization
    verbose=True
)

Optimizing input of length 10 to maximize output logits for " world"
Epoch 10/100, Loss: 5.6669 (Prob: 5.6607, Dist: 0.0061, Div: 0.0000)
Epoch 20/100, Loss: 4.8991 (Prob: 4.8925, Dist: 0.0066, Div: 0.0000)
Epoch 30/100, Loss: 4.6261 (Prob: 4.6196, Dist: 0.0065, Div: 0.0000)
Epoch 40/100, Loss: 4.5486 (Prob: 4.5421, Dist: 0.0065, Div: 0.0000)
Epoch 50/100, Loss: 4.5271 (Prob: 4.5206, Dist: 0.0065, Div: 0.0000)
Epoch 60/100, Loss: 4.5200 (Prob: 4.5135, Dist: 0.0065, Div: 0.0000)
Epoch 70/100, Loss: 4.5176 (Prob: 4.5111, Dist: 0.0065, Div: 0.0000)
Epoch 80/100, Loss: 4.5167 (Prob: 4.5102, Dist: 0.0065, Div: 0.0000)
Epoch 90/100, Loss: 4.5164 (Prob: 4.5099, Dist: 0.0065, Div: 0.0000)
Epoch 100/100, Loss: 4.5164 (Prob: 4.5098, Dist: 0.0065, Div: 0.0000)
